In [13]:
import janitor
import matplotlib.pyplot as plt
import missingno
import numpy as np
import pandas as pd
import pyreadr
import seaborn as sns
import session_info
import upsetplot

In [14]:
pima_indians_diabetes_url = "https://nrvis.com/data/mldata/pima-indians-diabetes.csv"

In [15]:
!wget -O ./data/pima-indians-diabetes.csv { pima_indians_diabetes_url } -q

In [16]:
df_diabetes = pd.read_csv(
    "./data/pima-indians-diabetes.csv",
    sep=',',
    names=[
        "pregnancies",
        "glucose",
        "blood_pressure",
        "skin_thickness",
        "insulin",
        "bmi",
        "diabetes_pedigree_function",
        "age",
        "outcome"]
)
df_diabetes

,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree_function,age,outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [17]:
base_url = "https://github.com/njtierney/naniar/raw/master/data/"
datasets_names = ("oceanbuoys", "pedestrian", "riskfactors")
extension = ".rda"

#### Download and load datasets

In [18]:
datasets_dfs = {}

for dataset_name in datasets_names:

    dataset_file = f"{ dataset_name }{ extension }"
    dataset_output_file = f"./data/{ dataset_file }"
    dataset_url = f"{ base_url }{ dataset_file }"
    
    !wget -q -O { dataset_output_file } { dataset_url }

    datasets_dfs[f"df_{ dataset_name }"] = pyreadr.read_r(dataset_output_file).get(dataset_name)

datasets_dfs.keys()

dict_keys(['df_oceanbuoys', 'df_pedestrian', 'df_riskfactors'])

In [19]:
locals().update(**datasets_dfs)
del datasets_dfs

In [34]:
df_oceanbuoys.shape, df_pedestrian.shape, df_riskfactors.shape

((736, 8), (37700, 9), (245, 34))

In [20]:
df_oceanbuoys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 736 entries, 0 to 735
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        736 non-null    float64
 1   latitude    736 non-null    float64
 2   longitude   736 non-null    float64
 3   sea_temp_c  733 non-null    float64
 4   air_temp_c  655 non-null    float64
 5   humidity    643 non-null    float64
 6   wind_ew     736 non-null    float64
 7   wind_ns     736 non-null    float64
dtypes: float64(8)
memory usage: 46.1 KB


In [30]:
df_oceanbuoys.head()

,year,latitude,longitude,sea_temp_c,air_temp_c,humidity,wind_ew,wind_ns
0,1997.0,0.0,-110.0,27.590000,27.15,79.599998,-6.4,5.4
1,1997.0,0.0,-110.0,27.549999,27.02,75.800003,-5.3,5.3
2,1997.0,0.0,-110.0,27.570000,27.00,76.500000,-5.1,4.5
3,1997.0,0.0,-110.0,27.620001,26.93,76.199997,-4.9,2.5
4,1997.0,0.0,-110.0,27.650000,26.84,76.400002,-3.5,4.1


In [29]:
df_pedestrian.head()

,hourly_counts,date_time,year,month,month_day,week_day,hour,sensor_id,sensor_name
0,883,2016-01-01 00:00:00,2016,January,1,Friday,0,2,Bourke Street Mall (South)
1,597,2016-01-01 01:00:00,2016,January,1,Friday,1,2,Bourke Street Mall (South)
2,294,2016-01-01 02:00:00,2016,January,1,Friday,2,2,Bourke Street Mall (South)
3,183,2016-01-01 03:00:00,2016,January,1,Friday,3,2,Bourke Street Mall (South)
4,118,2016-01-01 04:00:00,2016,January,1,Friday,4,2,Bourke Street Mall (South)


In [28]:
df_riskfactors.head()

,state,sex,age,weight_lbs,height_inch,bmi,marital,pregnant,children,education,...,smoke_100,smoke_days,smoke_stop,smoke_last,diet_fruit,diet_salad,diet_potato,diet_carrot,diet_vegetable,diet_juice
0,26,Female,49,190,64,32.68,Married,NaN,0,6,...,No,NaN,NaN,NaN,1095,261,104,156,521,12
1,40,Female,48,170,68,25.90,Divorced,NaN,0,5,...,No,NaN,NaN,NaN,52,209,52,0,52,0
2,72,Female,55,163,64,28.04,Married,NaN,0,4,...,No,NaN,NaN,NaN,36,156,52,24,24,24
3,42,Male,42,230,74,29.59,Married,NaN,1,6,...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,32,Female,66,135,62,24.74,Widowed,NaN,0,5,...,Yes,Everyday,Yes,NaN,-7,261,209,261,365,104


In [25]:
df_pedestrian.isna().sum()

hourly_counts    2548
date_time           0
year                0
month               0
month_day           0
week_day            0
hour                0
sensor_id           0
sensor_name         0
dtype: int64

In [27]:
df_riskfactors.isna().sum()

state                 0
sex                   0
age                   0
weight_lbs           10
height_inch           2
bmi                  11
marital               1
pregnant            215
children              0
education             1
employment            0
income                0
veteran               3
hispanic              2
health_general        0
health_physical       0
health_mental         0
health_poor         113
health_cover          0
provide_care          3
activity_limited      3
drink_any             2
drink_days          134
drink_average       135
smoke_100             2
smoke_days          128
smoke_stop          212
smoke_last          161
diet_fruit            8
diet_salad            8
diet_potato           8
diet_carrot           8
diet_vegetable        8
diet_juice            8
dtype: int64